In [25]:
import json
import requests
import flickrapi
import pandas as pd
from tqdm import tqdm
from vincenty import vincenty
import matplotlib.pyplot as plt
import time
import numpy as np

In [26]:
api_key = u'b766ae533b2843f748b174f53885eee4'
api_secret = u'ac3884caa9690723'

In [27]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')

In [28]:
min_lat = 59.800
max_lat = 69.910

# for Veronica
min_lon = 20.410
max_lon = 25.410


# for Ashish
#min_lon = 25.410
#max_lon = 30.410

n_lat = 6
n_lon = 2

start_lat = min_lat
start_lon = min_lon

lats = np.arange(min_lat, max_lat, (max_lat-min_lat)/n_lat)
lons = np.arange(min_lon, max_lon, (max_lon-min_lon)/n_lon)

bboxes = []

for i_lat, lat in enumerate(lats):
    if  (i_lat + 1 == n_lat):
        end_lat = max_lat
    else:
        end_lat = lats[i_lat+1]
            
    for i_lon, lon in enumerate(lons):
        if (i_lon + 1 == n_lon):
            end_lon = max_lon
        else:
            end_lon = lons[i_lon+1]
        
        bbox = (lon, lat, end_lon, end_lat)
        bboxes.append(bbox)

print(bboxes)
print(len(bboxes))

[(20.41, 59.8, 22.91, 61.485), (22.91, 59.8, 25.41, 61.485), (20.41, 61.485, 22.91, 63.17), (22.91, 61.485, 25.41, 63.17), (20.41, 63.17, 22.91, 64.855), (22.91, 63.17, 25.41, 64.855), (20.41, 64.855, 22.91, 66.54), (22.91, 64.855, 25.41, 66.54), (20.41, 66.54, 22.91, 68.22500000000001), (22.91, 66.54, 25.41, 68.22500000000001), (20.41, 68.22500000000001, 22.91, 69.91), (22.91, 68.22500000000001, 25.41, 69.91)]
12


In [29]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
photos = flickr.photos.search(per_page='250', bbox='20.410, 59.800, 30.920, 69.910', has_geo=1)
parsed = json.loads(photos.decode('utf-8'))

print(parsed)

id = parsed['photos']['photo'][0]['id']
geo = flickr.do_flickr_call('flickr.photos.geo.getLocation', photo_id=id)
geo_parsed = json.loads(geo.decode('utf-8'))
print(geo_parsed)
if 'locality' in geo_parsed['photo']['location']:
    print(geo_parsed['photo']['location']['locality']['_content'])
else:
    print('Unknown')


{'photos': {'page': 1, 'pages': 938, 'perpage': 250, 'total': '234258', 'photo': [{'id': '50991982323', 'owner': '63796798@N00', 'secret': '516cebe7e0', 'server': '65535', 'farm': 66, 'title': 'Tukkutori', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50992666801', 'owner': '63796798@N00', 'secret': '033c679dac', 'server': '65535', 'farm': 66, 'title': 'There is no way to peace, peace is the way', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50992084416', 'owner': '16391511@N00', 'secret': 'f513ac14b2', 'server': '65535', 'farm': 66, 'title': 'Ice-Covered Taivallahti', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50990491786', 'owner': '70192309@N03', 'secret': 'e3d7c6bf4a', 'server': '65535', 'farm': 66, 'title': 'Lightroom-852', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50990264196', 'owner': '91977610@N00', 'secret': 'a39ae4fc18', 'server': '65535', 'farm': 66, 'title': "St. Isaac's Cathedral", 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id'

{'photo': {'id': '50991982323', 'location': {'latitude': '60.191519', 'longitude': '24.973572', 'accuracy': '16', 'context': '0', 'locality': {'_content': 'Helsinki'}, 'county': {'_content': 'Helsinki'}, 'region': {'_content': 'Newland'}, 'country': {'_content': 'Finland'}, 'neighbourhood': {'_content': 'Hermanni'}}}, 'stat': 'ok'}
Helsinki


In [48]:
photos_set = []
for b in bboxes:
    # Let's count the pages!
    bbox = '{}, {}, {}, {}'.format(b[0], b[1], b[2], b[3])
    res = flickr.photos.search(per_page='250', bbox=bbox, has_geo=1)
    parsed = json.loads(res.decode('utf-8'))
    pages = int(parsed['photos']['pages'])
    print(bbox)
    for page in tqdm(range(1, pages + 1)):
        res = flickr.photos.search(page=page, per_page='250', bbox=bbox, has_geo=1)
        parsed = json.loads(res.decode('utf-8'))
        for p in parsed['photos']['photo']:
            photos_set.append(p)


  0%|                                                                                                                                                                                                                                              | 0/154 [00:00<?, ?it/s]

20.41, 59.8, 22.91, 61.485


  0%|                                                                                                                                                                                                                                             | 0/1045 [00:00<?, ?it/s]

22.91, 59.8, 25.41, 61.485


  0%|                                                                                                                                                                                                                                               | 0/72 [00:00<?, ?it/s]

...

  0%|                                                                                                                                                                                                                                               | 0/17 [00:00<?, ?it/s]

20.41, 68.22500000000001, 22.91, 69.91


  0%|                                                                                                                                                                                                                                               | 0/13 [00:00<?, ?it/s]

22.91, 68.22500000000001, 25.41, 69.91


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  1.88it/s]


In [49]:
len(photos_set)

414095

In [23]:
photos_set[10]

{'id': '50988487597',
 'owner': '41505253@N00',
 'secret': '980eaf04e3',
 'server': '65535',
 'farm': 66,
 'title': 'Late winter 2020-21',
 'ispublic': 1,
 'isfriend': 0,
 'isfamily': 0}

In [50]:
ids_df = pd.DataFrame(photos_set)
duplicateRowsDF = ids_df[ids_df.duplicated()]
print(len(ids_df))
len(duplicateRowsDF)

414095


373584

In [52]:
ids_df = ids_df.drop_duplicates()
duplicateRowsDF = ids_df[ids_df.duplicated()]
print(len(ids_df))
len(duplicateRowsDF)
ids_df.to_csv('ids.csv', index=False)

40511


In [57]:
ashish_df = df = pd.read_csv('ids_original.csv')

In [59]:
whole_df = pd.concat([ashish_df, ids_df])
duplicateRowsDF = whole_df[whole_df.duplicated()]
print(len(whole_df))
len(duplicateRowsDF)


82474


0

In [80]:
photos_all = ids_df.to_dict(orient='records')
print(photos_all[0:4])
print(len(photos_all))

[{'id': '50992175337', 'owner': '46915646@N06', 'secret': '71f03f815e', 'server': '65535', 'farm': 66, 'title': 'Spring', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50988380961', 'owner': '41505253@N00', 'secret': '5e36bd1410', 'server': '65535', 'farm': 66, 'title': 'Late winter 2020-21', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50988488177', 'owner': '41505253@N00', 'secret': '009ab112e5', 'server': '65535', 'farm': 66, 'title': 'Late winter 2020-21', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '50988380816', 'owner': '41505253@N00', 'secret': '960ab8b736', 'server': '65535', 'farm': 66, 'title': 'Late winter 2020-21', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}]
40511


In [78]:
for p in tqdm(photos_set):
    print(p)

  4%|█████████▋                                                                                                                                                                                                                    | 1759/40511 [00:00<00:02, 17496.57it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

 14%|██████████████████████████████▊                                                                                                                                                                                               | 5630/40511 [00:00<00:01, 18548.43it/s]


37870
37871
37872
37873
37874
37875
37876
37877
37878
37879
37880
37881
37882
37883
37884
37885
37886
37887
37888
37889
37890
37891
37892
37893
37894
37895
37896
37897
37898
37899
37900
37901
37902
37903
37904
37905
37906
37907
37908
37909
37910
37911
37912
37913
37914
37915
37916
37917
37918
37919
37920
37921
37922
37923
37924
37925
37926
37927
37928
37929
37930
37931
37932
37933
37934
37935
37936
37937
37938
37939
37940
37941
37942
37943
37944
37945
37946
37947
37948
37949
37950
37951
37952
37953
37954
37955
37956
37957
37958
37959
37960
37961
37962
37963
37964
37965
37966
37967
37968
37969
37970
37971
37972
37973
37974
37975
37976
37977
37978
37979
37980
37981
37982
37983
37984
37985
37986
37987
37988
37989
37990
37991
37992
37993
37994
37995
37996
37997
37998
37999
38000
38001
38002
38003
38004
38005
38006
38007
38008
38009
38010
38011
38012
38013
38014
38015
38016
38017
38018
38019
38020
38021
38022
38023
38024
38025
38026
38027
38028
38029
38030
38031
38032
38033
38034
38035
380

 24%|█████████████████████████████████████████████████████▏                                                                                                                                                                        | 9706/40511 [00:00<00:01, 19478.38it/s]

...

396394
396395
396396
396397
396398
396399
396400
396401
396402
396403
396404
396405
396406
396407
396408
396409
396410
396411
396412
396413
396414
396415
396416
396417
396418
396419
396420
396421
396422
396423
396424
396425
396426
396427
396428
396429
396430
396431
396432
396433
396434
396435
396436
396437
396438
396439
396440
396441
396442
396443
396444
396445
396446
396447
396448
396449
396450
396451
396452
396453
396454
396455
396456
396457
396458
396459
396460
396461
396462
396463
396464
396465
396466
396467
396468
396469
396470
396471
396472
396473
396474
396475
396476
396477
396478
396479
396480
396481
396482
396483
396484
396485
396486
396487
396488
396489
396490
396491
396492
396493
396494
396495
396496
396497
396498
396499
396500
396501
396502
396503
396504
396505
396506
396507
396508
396509
396510
396511
396512
396513
396514
396515
396516
396517
396518
396519
396520
396521
396522
396523
396524
396525
396526
396527
396528
396529
396530
396531
396532
396533
396534
396535
396536

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉           | 38482/40511 [00:01<00:00, 19472.76it/s]


409370
409371
409372
409373
409374
409375
409376
409377
409378
409379
409380
409381
409382
409383
409384
409385
409386
409387
409388
409389
409390
409391
409392
409393
409394
409395
409396
409397
409398
409399
409400
409401
409402
409403
409404
409405
409406
409407
409408
409409
409410
409411
409412
409413
409414
409415
409416
409417
409418
409419
409420
409421
409422
409423
409424
409425
409426
409427
409428
409429
409430
409431
409432
409433
409434
409435
409436
409437
409438
409439
409440
409441
409442
409443
409444
409445
409446
409447
409448
409449
409450
409451
409452
409453
409454
409455
409456
409457
409458
409459
409460
409461
409462
409463
409464
409465
409466
409467
409468
409469
409470
409471
409472
409473
409474
409475
409476
409477
409478
409479
409480
409481
409482
409483
409484
409485
409486
409487
409488
409489
409490
409491
409492
409493
409494
409495
409496
409497
409498
409499
409500
409501
409502
409503
409504
409505
409506
409507
409508
409509
409510
409511
40951

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40511/40511 [00:02<00:00, 19712.26it/s]

413539
413540
413541
413542
413543
413544
413545
413546
413547
413548
413549
413550
413551
413552
413553
413554
413555
413556
413557
413558
413559
413560
413561
413562
413563
413564
413565
413566
413567
413568
413569
413570
413571
413572
413573
413574
413575
413576
413577
413578
413579
413580
413581
413582
413583
413584
413585
413586
413587
413588
413589
413590
413591
413592
413593
413594
413595
413596
413597
413598
413599
413600
413601
413602
413603
413604
413605
413606
413607
413608
413609
413610
413611
413612
413613
413614
413615
413616
413617
413618
413619
413620
413621
413622
413623
413624
413625
413626
413627
413628
413629
413630
413631
413632
413633
413634
413635
413636
413637
413638
413639
413640
413641
413642
413643
413644
413645
413646
413647
413648
413649
413650
413651
413652
413653
413654
413655
413656
413657
413658
413659
413660
413661
413662
413663
413664
413665
413666
413667
413668
413669
413670
413714
413738
413739
413740
413741
413742
413743
413744
413745
413746
413747

In [76]:
# photo_id, lat, lon, name, locality, neighbourhood
photo_ids = []
lats = []
longs = []
names = []
localities = []
neighbourhoods = []
regions = []
for p in tqdm(photos_set):
    photo_id = p['id']
    geo = flickr.do_flickr_call('flickr.photos.geo.getLocation', photo_id=photo_id)
    parsed = json.loads(geo.decode('utf-8'))
    ## construct arrays of data
    photo_ids.append(photo_id)
    names.append(p['title'])
    lats.append(parsed['photo']['location']['latitude'])
    longs.append(parsed['photo']['location']['longitude'])
    neighbourhoods.append(parsed['photo']['location']['neighbourhood']['_content'])
    if 'locality' in parsed['photo']['location']:
        localities.append(parsed['photo']['location']['locality']['_content'])
    else:
        localities.append('Unknown')
    if 'region' in parsed['photo']['location']:
        regions.append(parsed['photo']['location']['region']['_content'])
    else:
        regions.append('Unknown')

  0%|                                                                                                                                                                                                                                            | 0/40511 [00:00<?, ?it/s]


TypeError: 'int' object is not subscriptable

## Data collection strategy:
We collect geographical information on all pictures taken in Finland inside the BBOX 20.410, 59.800, 30.920, 69.910.
Then, we test various clusterization algorithms to create ~500 nodes where each node represents a location.
For each location we can find ot the number of pictures taken, the most common names and tags. 
The edge is represented by distance calculated with Vincenty formula.
UPD We look into whole Europe (bbox is -25.31, 35.17, 40.96, 71.33)

In [19]:
df = pd.DataFrame({'id': photo_ids,
                  'name': names,
                  'lat': lats,
                  'lon': longs,
                  'locality': localities,
                  'region': regions,
                  'neighbourhood': neighbourhoods})

In [20]:
df.to_csv('photoset.csv', index=False)

In [21]:
df = pd.read_csv('photoset.csv')

In [22]:
duplicateRowsDF = df[df.duplicated()]
print(len(df))
len(duplicateRowsDF)

300


0

In [ ]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lon, df.lat))

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'), bbox=(20.410, 59.800, 30.920, 69.910))

ax = world[world.name == 'Finland'].plot(figsize=(10, 10),
    color='white', edgecolor='black')
ctx.add_basemap(ax)
gdf.plot(ax=ax, alpha=0.5, edgecolor='k')

plt.show()